In [2]:
from neo4j import GraphDatabase
from py2neo import Graph
import pandas as pd
import numpy as np

In [3]:
uri="neo4j+s://1c8f07f5.databases.neo4j.io"
username="neo4j"
password="uycJUUxlA1SDUncKIgImCp_DT8hASOiqSQ-V7pnLRCY"

In [4]:
#get the datasets
user_ratings = pd.read_csv('data\\ratings_train.csv')
tmdb = pd.read_csv('data\\tmdb_movies_data.csv')

In [5]:
# Connect to the Neo4j database
graph = GraphDatabase.driver(uri, auth=(username, password))

# Create a session
session = graph.session()

for _, i in user_ratings.iterrows():
    id = i['userId']
    name = i['movieTitle']
    df = tmdb[tmdb['original_title'] == name]

    original_title = df["original_title"].values[0]
    release_date = df["release_date"].values[0]
    popularity = df["popularity"].values[0]

    query_to_create_user_add_rating = """
    MERGE (u:User {id: $id})
    WITH u
    MATCH (m:Movie {original_title: $original_title, release_date: $release_date, popularity: $popularity})
    MERGE (u)-[:watched]->(m)
    """

    parameters_genre = {
                "original_title": original_title,
                "release_date": release_date,
                "popularity": popularity,
                "id": id
            }
    session.run(query_to_create_user_add_rating, parameters_genre)

# Close the session
session.close()

# Close the Neo4j connection
graph.close()
